In [ ]:
%load_ext dotenv
%dotenv ../src/.env

# Download Data from Yahoo Finance

References:

+ [yfinance documentation](https://pypi.org/project/yfinance/)
+ [Tutorial in geeksforgeeks.org](https://www.geeksforgeeks.org/get-financial-data-from-yahoo-finance-with-python/)


In [ ]:
import os
os.getenv('LOG_DIR')

'../logs/'